<a href="https://colab.research.google.com/github/JasminiSantos/Trabalho-5-Bag-of-Words/blob/main/Trabalho_5_Bag_of_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Jasmini Rebecca Gomes dos Santos**

# Extração de sentenças das urls de websites com temas relacionados à linguagem de processamento natural

In [8]:
# Jasmini Rebecca Gomes dos Santos
# Obs: Execute todas as células para conseguir rodar o código corretamente
'''
Sua tarefa será  gerar a matriz termo documento, dos documentos recuperados da internet e 
imprimir esta matriz na tela. Para tanto: 
a) Considere que todas as listas de sentenças devem ser transformadas em listas de vetores, 
onde cada item será uma das palavras da sentença. 
b) Todos  os  vetores  devem  ser  unidos  em  um  corpus  único  formando  uma  lista  de  vetores, 
onde cada item será um lexema.  
c) Este único corpus será usado para gerar o vocabulário. 
d) O  resultado  esperado  será  uma  matriz  termo  documento  criada  a  partir  da  aplicação  da 
técnica bag of Words em todo o corpus.  
'''

from __future__ import unicode_literals, print_function
from spacy.lang.en import English 
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import traceback
import sys
import unicodedata
import spacy
from spacy.pipeline import EntityRuler
from spacy import displacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")

def split_in_sentences(text):
    doc = nlp(text)
    return [str(sent).strip() for sent in doc.sents]

def remove_invalid_characters(text):
  text = text.strip().split("\n")
  text_array = []
  for i in range(0, len(text) - 1):
    if text[i] != "" and re.search('[a-z0-9]', text[i]):
      text[i] = unicodedata.normalize("NFKD", text[i])
      text[i] = text[i].strip()
      text_array.append(text[i])
  text = "\n".join(text_array)
  return text

def extract_text_from_url(url):
    try:
      req = Request(url, headers={'User-Agent': 'XYZ/3.0'})
      html = urlopen(req, timeout=10).read()
      soup = BeautifulSoup(html, 'lxml') 
      invalid_tags = ['[document]','noscript','meta','head', 'input','script', 'style', 'svg']
      for tag in soup.find_all(invalid_tags):
          tag.extract()
      text = soup.getText()
      text = remove_invalid_characters(text)
      text_array = split_in_sentences(text)
      text = "\n".join(text_array)
      text = remove_invalid_characters(text)
      return text
    except:
        print(traceback.format_exc())
        sys.exit(-1)

def find_number_words(text):
  words_array = re.findall(r"[\w']+", text)
  return len(words_array)
  
def description_of_text_from_url(url):
  text = extract_text_from_url(url)
  sentence_array = text.split("\n")
  print("***************************************")
  print("Url: " + url)
  print("Number of words: " + str(find_number_words(text)))
  print("Number of sentences: " + str(len(sentence_array)))
  print("List of sentences: ")
  print(sentence_array)
  print("***************************************")
  
url1 = "https://www.ibm.com/cloud/learn/natural-language-processing"
url2 = "https://www.dominodatalab.com/blog/natural-language-in-python-using-spacy"
url3 = "https://www.analyticsvidhya.com/blog/2020/12/understanding-text-classification-in-nlp-with-movie-review-example-example/"
url4 = "https://blogs.commons.georgetown.edu/cctp-607-spring2019/2019/02/27/natural-language-processing-google-translate/"
url5 = "https://towardsai.net/p/nlp/natural-language-processing-nlp-with-python-tutorial-for-beginners-1f54e610a1a0"
urls = [url1, url2, url3, url4, url5]

for url in urls:
  description_of_text_from_url(url)

***************************************
Url: https://www.ibm.com/cloud/learn/natural-language-processing
Number of words: 1795
Number of sentences: 99
List of sentences: 
['Skip to content', 'IBM Cloud Learn Hub', 'What is Natural Language Processing?', 'Natural Language Processing (NLP)', 'By:', 'IBM Cloud Education', '2 July  2020', 'Artificial intelligence', 'What is natural language processing?', 'NLP tasks', 'NLP tools and approaches', 'NLP use cases', 'Natural language processing and IBM Watson', 'Jump to ...', 'What is natural language processing?', 'NLP tasks', 'NLP tools and approaches', 'NLP use cases', 'Natural language processing and IBM Watson', 'Natural Language Processing (NLP)', 'Natural language processing strives to build machines that understand and respond to text or voice data—and respond with text or speech of their own—in much the same way humans do.', 'What is natural language processing?', 'Natural language processing (NLP) refers to the branch of computer scie

# Técnica Bag of Words

In [9]:
contractions = {
"ain't" :"am not",
"aren't": "are not",
"can't": "cannot have",
"cause": "because",
"could've": "could have",
"couldn't": "could not",
"didn't": "did not",
"doesn't": "does not",
"doesn’t": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",    
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"it’s": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"musn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"they'd":  "they would",
"they'd've": "they would have",
"they'll": "they will",    
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": "you",
" ur ": "your",
" n ": " and "
}

def replace_contractions(x):
    for word in x.split():
        if word in contractions:
          x = x.replace(word, contractions[word])
    return x

def print_matrix(matrix, vocab):
  head = "Doc"
  line = ""
  table = ""
  for i in range(len(matrix)):
    head = head + " " + str(i+1)
  head = head + "\n"
  head = head + "-----------------"
  print(head)  
  for i in range(len(vocab)):
    line = vocab[i] + " "
    for j in range(len(matrix)):
      line = line + str(matrix[j][i]) + " "
    table = table + line + "\n"
  print(table)  

def get_vocabulary_from_texts(urls):
  vocab = []
  for url in urls:
    text = extract_text_from_url(url)
    text = replace_contractions(text)
    words = re.findall(r'\w+',text)
    for word in words:
      if word not in vocab:
        vocab.append(word)
  return vocab

def get_all_texts(urls):
  documents = []
  for url in urls:
    text = extract_text_from_url(url)
    text = replace_contractions(text)
    words = re.findall(r'\w+',text)
    documents.append(words)
  return documents

print(get_vocabulary_from_texts(urls))

['Skip', 'to', 'content', 'IBM', 'Cloud', 'Learn', 'Hub', 'What', 'is', 'Natural', 'Language', 'Processing', 'NLP', 'By', 'Education', '2', 'July', '2020', 'Artificial', 'intelligence', 'natural', 'language', 'processing', 'tasks', 'tools', 'and', 'approaches', 'use', 'cases', 'Watson', 'Jump', 'strives', 'build', 'machines', 'that', 'understand', 'respond', 'text', 'or', 'voice', 'data', 'with', 'speech', 'of', 'their', 'own', 'in', 'much', 'the', 'same', 'way', 'humans', 'do', 'refers', 'branch', 'computer', 'science', 'more', 'specifically', 'artificial', 'AI', 'concerned', 'giving', 'computers', 'ability', 'spoken', 'words', 'human', 'beings', 'can', 'combines', 'computational', 'linguistics', 'rule', 'based', 'modeling', 'statistical', 'machine', 'learning', 'deep', 'models', 'Together', 'these', 'technologies', 'enable', 'process', 'form', 'its', 'full', 'meaning', 'complete', 'speaker', 'writer', 's', 'intent', 'sentiment', 'drives', 'programs', 'translate', 'from', 'one', 'anot

In [10]:
def generate_bag_of_words(urls):
  documents = get_all_texts(urls)
  vocab = get_vocabulary_from_texts(urls)
  matrix = []
  for document in documents:
    array = []
    for word in vocab:
      if word not in document:
        array.append(0)
      else:
        array.append(document.count(word))
    matrix.append(array)
  print_matrix(matrix, vocab)
generate_bag_of_words(urls)

Doc 1 2 3 4 5
-----------------
Skip 1 1 0 1 0 
to 54 65 54 24 128 
content 1 2 0 2 4 
IBM 11 0 0 0 0 
Cloud 3 2 0 1 10 
Learn 1 0 0 0 4 
Hub 1 0 0 0 0 
What 7 1 2 0 3 
is 28 26 49 13 104 
Natural 15 2 1 3 28 
Language 8 2 3 4 30 
Processing 4 1 1 3 25 
NLP 34 5 26 1 52 
By 1 2 2 0 4 
Education 1 0 0 0 3 
2 1 3 1 2 9 
July 1 0 0 0 1 
2020 1 0 4 0 4 
Artificial 1 0 2 2 3 
intelligence 4 0 2 0 3 
natural 10 21 1 0 12 
language 33 25 8 9 20 
processing 11 3 2 0 18 
tasks 8 3 0 0 5 
tools 7 0 0 0 0 
and 79 45 65 19 128 
approaches 4 1 0 0 1 
use 10 11 6 3 30 
cases 3 4 0 0 6 
Watson 15 0 0 0 0 
Jump 1 0 0 0 0 
strives 1 0 0 0 0 
build 1 2 1 0 2 
machines 1 0 1 0 1 
that 20 27 15 9 49 
understand 5 4 8 1 11 
respond 5 0 0 0 0 
text 32 37 22 1 55 
or 21 8 7 2 25 
voice 11 0 2 0 0 
data 16 11 14 1 28 
with 18 23 9 5 61 
speech 10 1 4 2 3 
of 50 56 61 12 175 
their 6 2 1 2 5 
own 3 0 0 0 0 
in 31 68 30 13 84 
much 3 2 0 0 2 
the 60 120 83 47 339 
same 5 0 0 1 8 
way 4 1 1 3 0 
humans 2 0 1 0 4

# Dataframe da matriz de palavras extraídas dos textos, listando a frequência de termos por texto

In [11]:
import pandas as pd
import numpy as np
def generate_bag_of_words_dataframe(urls):
  documents = get_all_texts(urls)
  vocab = get_vocabulary_from_texts(urls)
  dictionaries = []
  for document in documents:
    dictionary = dict.fromkeys(document, 0)
    for word in dictionary:
      dictionary[word] = document.count(word)
    dictionaries.append(dictionary)
  df = pd.DataFrame(dictionaries)
  df = df.replace(np.nan, 0)
  df = df.apply(np.int64)
  display(df)
generate_bag_of_words_dataframe(urls)

,Skip,to,content,IBM,Cloud,Learn,Hub,What,is,Natural,...,bounce,rate,Advertisement,customized,Others,uncategorized,Hit,enter,ESC,close
0,1,54,1,11,3,1,1,7,28,15,...,0,0,0,0,0,0,0,0,0,0
1,1,65,2,0,2,0,0,1,26,2,...,0,0,0,0,0,0,0,0,0,0
2,0,54,0,0,0,0,0,2,49,1,...,0,0,0,0,0,0,0,0,0,0
3,1,24,2,0,1,0,0,0,13,3,...,0,0,0,0,0,0,0,0,0,0
4,0,128,4,0,10,4,0,3,104,28,...,1,1,3,1,2,1,1,1,1,1
